In [ ]:
from sklearn.model_selection import GridSearchCV
import timeit # for timing functions
import time # for getting time related data
import numpy as np
import scipy.io as sio
from sklearn.svm import SVC
import pandas as pd
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
X = pd.read_csv('pca_train_txt.csv', header=None)
Y = pd.read_csv('pca_train_img_vgg.csv', header=None)
train_labels = np.load('train_labels.npy',allow_pickle=True)

In [ ]:
X_test = np.load('pca_test_txt.npy',allow_pickle=True)
Y_test = np.load('pca_test_img_vgg.npy',allow_pickle=True)
test_labels = np.load('test_labels.npy',allow_pickle=True)

In [ ]:
# Loanding W
IVA_M_EMK = sio.loadmat('W_IVA_G.mat')
W_IVA_M_EMK = IVA_M_EMK['W_IVA_G']

In [ ]:
# Split W into two matrix to get a W1 for X and a W2  for Y.
W1_IVA_M_EMK, W2_IVA_M_EMK = np.split(W_IVA_M_EMK, 2, axis = 2)
W1_IVA_M_EMK = np.squeeze(W1_IVA_M_EMK, axis = 2)
W2_IVA_M_EMK = np.squeeze(W2_IVA_M_EMK, axis = 2)

In [ ]:
# Generate S1 and S2

S1_tr = W1_IVA_M_EMK @ np.transpose(X) #Text Components
S2_tr = W2_IVA_M_EMK @ np.transpose(Y) #Image Components

S1_test = W1_IVA_M_EMK @ np.transpose(X_test) #Text Components
S2_test = W2_IVA_M_EMK @ np.transpose(Y_test) #Image Components

In [ ]:
# Data Fusion - Averaging
df_IVA_M_EMK = (S1_tr.T + S2_tr.T)/2

df_test_IVA_M_EMK = (S1_test.T + S2_test.T)/2

In [ ]:
# SVC hyperparams to optimize

parameters = {'C':[0.001,0.01, 0.1, 1,10],
                'kernel':['rbf', 'poly', 'sigmoid']}

scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'binary'),
           'recall': make_scorer(recall_score, average = 'binary'),
           'f1': make_scorer(f1_score, average = 'binary')}

In [ ]:
# CV
cv = StratifiedKFold(n_splits=5, shuffle=True)
svm = SVC(gamma='scale',random_state=1)
svm_GS = GridSearchCV(svm, param_grid = parameters, scoring = scoring, refit='f1', cv=cv)
svm_GS.fit(df_IVA_M_EMK,train_labels)

In [ ]:
pred_svm = svm_GS.predict(df_test_IVA_M_EMK)

In [ ]:
iva_clc_rpt = classification_report(test_labels, pred_svm, target_names=['Real', 'Fake'], digits=4)
iva_clc_rpt